# Imports

In [1]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

# Trainer class

In [2]:
def create_Model():

    model = tf.keras.models.Sequential([
        # Your Code Here
        tf.keras.layers.Conv2D(16, (3,3), activation = 'relu', data_format = 'channels_last', input_shape = (150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        # The second convolution
        tf.keras.layers.Conv2D(32, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # The third convolution
        tf.keras.layers.Conv2D(64, (3,3), activation = 'relu'),
        tf.keras.layers.MaxPooling2D(2,2),
        # Flatten the results to feed into a DNN
        tf.keras.layers.Flatten(),
        # 512 neuron hidden layer
        tf.keras.layers.Dense(128, activation = 'relu'),
        tf.keras.layers.Dense(1, activation = 'sigmoid')
    ])
    
    from tensorflow.keras.optimizers import RMSprop

    model.compile(loss = 'binary_crossentropy',
                optimizer = RMSprop(lr = 0.001),
                metrics = ['acc'])

    return model

In [3]:
def fit_model(model, train_dir, val_dir):

    DESIRED_ACCURACY = 0.99

    class myCallback(tf.keras.callbacks.Callback):
        def on_epoch_end(self, epoch, logs={}):
            if (logs.get('acc')>DESIRED_ACCURACY):
              print('\nReached 99% accuracy so cancelling training!')
              self.model.stop_training = True

    from tensorflow.keras.preprocessing.image import ImageDataGenerator

    datagen = ImageDataGenerator(rescale=1/255) # bring every pixel value between 0 and 1
    train_generator = datagen.flow_from_directory(
        train_dir,
        target_size = (150, 150),  # All images will be resized to 150x150
        batch_size = 64,
        class_mode = 'binary')
    
    val_generator = datagen.flow_from_directory(
        val_dir,
        target_size = (150, 150),  # All images will be resized to 150x150
        batch_size = 16,
        class_mode = 'binary')

    callbacks = myCallback()

    history = model.fit(
            train_generator,
            steps_per_epoch = 8,
            validation_data = val_generator,
            validation_steps = 4,
            epochs = 10,
            verbose = 1, 
            callbacks = [callbacks])
    model.reset_metrics()
    # model fitting
    return model

In [4]:
def predict_images(image_paths, model):
    from keras.preprocessing import image as kimg
    from PIL import Image as pimg
    for image_path in image_paths:
        plt.figure()
        plt.imshow(pimg.open(image_path)) 
        plt.show() 
        img = kimg.load_img(image_path, target_size = (150, 150))
        img_array = kimg.img_to_array(img)
        img_batch = np.expand_dims(img_array, axis = 0)
        img_stacked = np.vstack([img_batch])
        classes = model.predict(img_stacked)
        if (classes[0] < 0.5):
            print('This mathematical function is linear')
        else:
            print('This mathematical function is squared')
        